# Mission to Mars

In [52]:
# dependencies 
from bs4 import BeautifulSoup as bs
from requests import get 
import requests as req
from pprint import pprint
import pandas as pd
import splinter
from splinter import Browser
import pymongo
import time

In [42]:
!which chromedriver

/usr/local/bin/chromedriver


## Initialize PyMongo DB 

In [53]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [54]:
# Define database and collection
db = client.mars_db
collection = db.mars_data

## Martian News from NASA

In [45]:
# first url to scrape and retrieve page with the requests module
url = "https://mars.nasa.gov/news/" 
response = req.get(url)

# creating bs object
html_soup = bs(response.text, 'html.parser')

# extract necessary info 
news_title = html_soup.find('div', class_='content_title').text.strip()
news_p = html_soup.find('div', class_="rollover_description_inner").text.strip()

print('The title: "{}"'.format(news_title))
print("-"*100)
print('The paragraph text: "{}"'.format(news_p))

    
# Dictionary to be inserted as a MongoDB document
mars_news = {
    "Title":news_title,
    "Text": news_p
}
collection.insert_one(mars_news)

The title: "Opportunity Hunkers Down During Dust Storm"
----------------------------------------------------------------------------------------------------
The paragraph text: "It's the beginning of the end for the planet-encircling dust storm on Mars. But it could still be weeks, or even months, before skies are clear enough for NASA's Opportunity rover to recharge its batteries and phone home."


## JPL Space Images

In [46]:
# Navigation 
executable_path = {'executable_path' : '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)
url_2 = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(url_2)

In [47]:
# second url (JPL) 
url_2 = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(url_2)
# extract info 
time.sleep(1)
browser.click_link_by_partial_text('FULL IMAGE')
time.sleep(1)
expand = browser.find_by_css('a.fancybox-expand')
expand.click()
time.sleep(1)
# bs object
jpl_html = browser.html
jpl_soup = bs(jpl_html, 'html.parser')
# extract info and append img pathway to jpl url 
img_relative = jpl_soup.find('img', class_='fancybox-image')['src']
image_path = f'https://www.jpl.nasa.gov{img_relative}'
print(image_path)

# update collection in mongodb 
jpl_img = {
    "url": image_path
}
collection.insert_one(jpl_img)

https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA16715_ip.jpg


## Martian Weather from Twitter

In [62]:
# take mars twitter weather url, add request module and turn it into a bs object
weather_url = 'https://twitter.com/marswxreport?lang=en'
browser.visit(weather_url)
time.sleep(1)
weather_html = browser.html
weather_soup = bs(weather_html, 'html.parser')

tweets = weather_soup.find('ol', class_='stream-items')
tw_weather = tweets.find('p', class_="tweet-text").text
print(tw_weather)


# update collection in mongodb 
tweet = {
    "Tweet": tw_weather
}
collection.insert_one(tweet)

A view of this morning’s launch of the Parker #SolarProbe launch from about 3 miles awayhttps://youtu.be/hDhIane1Rso 


## Martian Facts

In [68]:
# use space facts url to scrape info. 
url_4 = req.get("https://space-facts.com/mars/")
# constract pandas
facts_table = pd.read_html(url_4.text)
facts_table
# build data frame
df = facts_table[0]
df

# update collection in mongodb 
# mars_facts = {
#     "Facts": df
# }
# collection.insert_one(mars_facts)

,0,1
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.42 x 10^23 kg (10.7% Earth)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.52 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-153 to 20 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [69]:
# use fluffy Pandas to convert table data into a html table string 
mars_html_tablestring = df.to_html()
mars_html_tablestring.replace('\n', '')

'<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th>0</th>      <th>1</th>    </tr>  </thead>  <tbody>    <tr>      <th>0</th>      <td>Equatorial Diameter:</td>      <td>6,792 km</td>    </tr>    <tr>      <th>1</th>      <td>Polar Diameter:</td>      <td>6,752 km</td>    </tr>    <tr>      <th>2</th>      <td>Mass:</td>      <td>6.42 x 10^23 kg (10.7% Earth)</td>    </tr>    <tr>      <th>3</th>      <td>Moons:</td>      <td>2 (Phobos &amp; Deimos)</td>    </tr>    <tr>      <th>4</th>      <td>Orbit Distance:</td>      <td>227,943,824 km (1.52 AU)</td>    </tr>    <tr>      <th>5</th>      <td>Orbit Period:</td>      <td>687 days (1.9 years)</td>    </tr>    <tr>      <th>6</th>      <td>Surface Temperature:</td>      <td>-153 to 20 °C</td>    </tr>    <tr>      <th>7</th>      <td>First Record:</td>      <td>2nd millennium BC</td>    </tr>    <tr>      <th>8</th>      <td>Recorded By:</td>      <td>Egyptian astronomers</td>    <

##  Martian Hemispheres

In [71]:
# use USGS astrogeology url to scrape images of martian hemispheres
mars_hemisphere_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
# empty list to hold image urls as dictionary
hemi_dicts = []

for i in range(1,9,2):
    hemi_dict = {}
    
    browser.visit(mars_hemisphere_url)
    time.sleep(1)
    hemispheres_html = browser.html
    hemispheres_soup = bs(hemispheres_html, 'html.parser')
    hemi_name_links = hemispheres_soup.find_all('a', class_='product-item')
    hemi_name = hemi_name_links[i].text.strip('Enhanced')
    
    detail_links = browser.find_by_css('a.product-item')
    detail_links[i].click()
    time.sleep(1)
    browser.find_link_by_text('Sample').first.click()
    time.sleep(1)
    browser.windows.current = browser.windows[-1]
    hemi_img_html = browser.html
    browser.windows.current = browser.windows[0]
    browser.windows[-1].close()
    
    hemi_img_soup = bs(hemi_img_html, 'html.parser')
    hemi_img_path = hemi_img_soup.find('img')['src']

    print(hemi_name)
    hemi_dict['title'] = hemi_name.strip()
    
    print(hemi_img_path)
    hemi_dict['img_url'] = hemi_img_path

    hemi_dicts.append(hemi_dict)
    
hemispheres = {
    "Hemisphere":hemi_name,
    "Hemisphere Img": hemi_img_path
}
    
collection.insert_one(hemispheres)


Cerberus Hemisphere 
https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg
Schiaparelli Hemisphere 
https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg
Syrtis Major Hemisphere 
https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg
Valles Marineris Hemisphere 
https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg
